In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

train_ori = pd.read_csv('../input/ventilator-pressure-prediction/train.csv', index_col=None, header=0)


def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()

    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    # df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    # df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    # df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    # df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    # df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    # df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    # df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    # df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    # df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    # df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    # df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    # df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    # df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    # df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    # df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)

    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    # df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')

    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    # df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df.drop(['u_in_lag1'], axis=1)
    # df.drop(['u_out_lag1'], axis=1)
    # df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    # df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']

    # df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    # df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

    # df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    # df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    # df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    # df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross'] = df['u_in'] * df['u_out']
    df['cross2'] = df['time_step'] * df['u_out']

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df


train = add_features(train_ori)
#%%

targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
train = train.values
#%%

train_x_other = train[:, :-15]
train_x_rc = train[:, -15:]
#%%

scale = StandardScaler()
scale.fit(train_x_other)
train_x_other = scale.transform(train_x_other)
train_x_other = train_x_other.reshape(-1, 80, train_x_other.shape[-1])
train_x_rc = train_x_rc.reshape(-1, 80, train_x_rc.shape[-1])

#%%

np.save('./train_x_rc.npy', train_x_rc)
np.save('./train_x_other.npy', train_x_other)
np.save('./target.npy', targets)

In [2]:
#%%

from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.keras import optimizers
from tensorflow.python.keras import activations
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import initializers
from tensorflow.python.keras import backend
from tensorflow.python.keras import metrics
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.model_selection import train_test_split

#%%

train_x_rc = np.load('./train_x_rc.npy')
train_x_other = np.load('./train_x_other.npy')
targets = np.load('./target.npy')
print(train_x_other.shape)
print(train_x_rc.shape)
print(targets.shape)

#%%

rc_num = 15
ox = 10
train_x = np.concatenate([train_x_rc, train_x_other], axis=-1)
x_train, x_test, y_train, y_test = train_test_split(train_x, targets, test_size=0.15, random_state=43)
x_train_rc = x_train[:, :, :rc_num][:, 0]
x_train_ox = x_train[:, :, -ox:]
x_test_rc = x_test[:, :, :rc_num][:, 0]
x_test_ox = x_test[:, :, -ox:]

In [3]:
class ExpandTileLayer(layers.Layer):
    def __init__(self):
        super(ExpandTileLayer, self).__init__()
    def call(self, inputs, *args, **kwargs):
        return backend.tile(backend.expand_dims(inputs, axis=-2), (1, 80, 1))

In [4]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():


    #%%

    rc_input = keras.Input(shape=(train_x_rc.shape[-1], ),
                           dtype='int32',
                           name='rc_input_layer')
    other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
                                dtype='float32',
                                name='other_x_input')
    rc_embedding_layer = layers.Dense(units=32,
                                      use_bias=False, name='rc_embed_layer')
    rc_embedding = rc_embedding_layer(rc_input)

    # 维度拓展
    rc_embedding = ExpandTileLayer()(rc_embedding)

    x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
    ox = layers.Bidirectional(layers.LSTM(units=128,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(x_input)

    ox = layers.Bidirectional(layers.LSTM(units=64,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(ox)
    ox = layers.Dense(units=128,
                      kernel_initializer=initializers.initializers_v2.GlorotUniform())(ox)
    # if hp.Choice('d2_activation', ['prelu', 'elu']) == 'prelu':
    lstm_output = layers.PReLU()(ox)
    # else:
    #     lstm_output = layers.ELU()(ox)

    conv1d_1 = layers.Conv1D(filters=256,
                             kernel_size=11,
                             padding='same', use_bias=False)(x_input)
    conv1d_1 = layers.BatchNormalization()(conv1d_1)
    conv1d_1 = layers.ReLU()(conv1d_1)
    conv1d_2 = layers.Conv1D(filters=64, padding='same', use_bias=False,
                             kernel_size=5)(conv1d_1)
    conv1d_2 = layers.BatchNormalization()(conv1d_2)
    conv1d_output = layers.ReLU()(conv1d_2)

    output = layers.Concatenate(axis=-1)([lstm_output, conv1d_output])
    output = layers.Dense(units=256,
                          activation=activations.relu,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
    output = layers.Dense(units=1,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

    my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])

    my_callbacks = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1),
                    callbacks.EarlyStopping(monitor='val_loss', patience=31, verbose=1, mode='min', restore_best_weights=True)]
    my_model.compile(loss=losses.MeanAbsoluteError(),
                     optimizer=optimizers.Adam(),
                     metrics=[metrics.MeanAbsoluteError()])

In [ ]:
my_model.fit(x=[x_train_rc, x_train_ox], y=y_train,
             epochs=128,
             batch_size=256,
             validation_data=([x_test_rc, x_test_ox], y_test),
             callbacks=my_callbacks,
             verbose=1)


In [ ]:
my_model.save('./my_model_1_o_train')

In [ ]:
test_callbacks = [
                  callbacks.EarlyStopping(monitor='loss', patience=25, restore_best_weights=True, mode='min'),
                  callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=8)]
my_model.fit(x=[x_test_rc, x_test_ox], y=y_test,
             batch_size=256,
             epochs=48,
             verbose=1)


In [ ]:
my_model.save('./my_model_1_alld')